In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
#read open data from ANAC
link1 =  '/Users/andreza/reproducibility_transportation_hubs-early_warning_surveillance_systems/data/aero_anac_2017_2023.parquet'

df2 = pd.read_parquet(link1, engine='pyarrow')

In [2]:
muni = pd.read_csv('/Users/andreza/reproducibility_transportation_hubs-early_warning_surveillance_systems/data/DTB_BRASIL_MUNICIPIO.csv',sep=';')

In [3]:
df2 = df2[df2['nat'] == 'DOMÉSTICA']

In [4]:
df2.ano = df2.ano.astype(int)
df2.mes = df2.mes.astype(int)

In [5]:
busca = df2.index[df2['co_muni_des'] == 'SÃO LOURENÇO']
df2.loc[busca, 'co_muni_des'] = 3163706
busca = df2.index[df2['co_muni_des'] == 'COLATINA']
df2.loc[busca, 'co_muni_des'] = 3201506

In [6]:
df2.co_muni_ori = df2.co_muni_ori.astype(int)
df2.co_muni_des = df2.co_muni_des.astype(int)

In [7]:
data_2022 = df2[df2['ano'] == 2022]

In [ ]:
# Create a dictionary to store adjacency matrices (origin vs destinations) for each month
adjacency_matrices = {}

# Loop through each month
for month in range(1, 13):
    # Filter the data for the current month
    data_month = data_2022[data_2022['mes'] == month]
    
    # Create the adjacency matrix for the current month
    adjacency_matrix = data_month.pivot_table(index='co_muni_ori', columns='co_muni_des', values='aero_pass', aggfunc='sum', fill_value=0)
    
    # Save the adjacency matrix for the current month in the dictionary
    adjacency_matrices[month] = adjacency_matrix

# Display or save the adjacency matrices for each month
for month, adjacency_matrix in adjacency_matrices.items():
    print(f"Adjacency matrix for month {month}:\n{adjacency_matrix}\n")

In [ ]:
# check the number of rows and columns
df_matrix_month_1 = adjacency_matrices[1]

# Get the number of rows and columns
num_rows, num_columns = df_matrix_month_1.shape

# Print the number of rows and columns
print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_columns}")

In [ ]:
#to make all 5570 municipalities present in the matrices columns
#test month_1
df3 = muni
lst_2022 = set(df3['Código Município Completo']) - set(df_matrix_month_1.columns)

In [ ]:
# for value in lst:
# #     for matrix in adjacency_matrices.items():
    
#     lis = [0]*len(df_matrix_month_1)

#     d = {value: lis}
#     dtf = pd.DataFrame(data=d)
#     dtf.set_index(df_matrix_month_1.index)
    
#     df_matrix_month_1 = df_matrix_month_1.join(dtf)

#trying to make a parallel
import pandas as pd
from multiprocessing import Pool, cpu_count

def process_value(value):
    lis = [0] * len(df_matrix_month_1)
    d = {value: lis}
    dtf = pd.DataFrame(data=d)
    dtf.set_index(df_matrix_month_1.index)
    return dtf

if __name__ == '__main__':
    # Number of process and cpus
    num_process = cpu_count()

    # pool of process
    pool = Pool(processes=num_process)

    # Aplpy function processar_valor for each value in lst
    results = pool.map(process_value, lst_2022)

    # Close pool and make resources available
    pool.close()
    pool.join()

    # Join results to the original DataFrame
    for dtf in results:
        df_matrix_month_1 = df_matrix_month_1.join(dtf)

In [ ]:
# check the number of rows and columns
# df_matrix_month_1 = adjacency_matrices[1]

# Get the number of rows and columns
num_rows, num_columns = df_matrix_month_1.shape

# Print the number of rows and columns
print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_columns}")

In [ ]:
#to make all 5570 municipalities present in the matrices rowns
#test month_1
lst_index = set(df3['Código Município Completo']) - set(df_matrix_month_1.index)

In [ ]:
for value in lst_index:
    df_matrix_month_1.loc[value] = [0]*5570

In [ ]:
df_matrix_month_1 = df_matrix_month_1.fillna(0)
df_matrix_month_1

In [ ]:
#adaptando o codigo para iterar em todos os meses

In [ ]:
for key, matrix in adjacency_matrices.items():
    columns_to_add = set(df3['Código Município Completo']) - set(matrix.columns) # Get columns to add for the specific matrix
    
    # Determine columns that are missing in the matrix
    missing_columns = list(columns_to_add)
    
    # Create a DataFrame with missing columns filled with zeros
    new_columns_df = pd.DataFrame({col: [0] * len(matrix) for col in missing_columns}, index=matrix.index)
    
    # Join the new DataFrame to the specific matrix in adjacency_matrices
    adjacency_matrices[key] = matrix.join(new_columns_df)

In [ ]:
# Get the number of rows and columns
for month in range(1, 13):
    num_rows, num_columns = adjacency_matrices[month].shape
    
    # Print the number of rows and columns
    print(f"Month:{month}, Number of rows: {num_rows}, Number of columns: {num_columns}")

In [ ]:
%%time
for month in range(1, 13):
    lst_index = set(df3['Código Município Completo']) - set(adjacency_matrices[month].index)
    for value in lst_index:
        adjacency_matrices[month].loc[value] = [0]*5570

In [ ]:
# dat_air = dat_air.fillna(0)
# dat_air

# Version 2
## a new approach to work with a template/model df and generate all matrices

In [9]:
%%time
#genrate a model data frame with all pairs/combinations of municipalities 5570x5570
df3 = muni
list_cod_munic_completo = df3['Código Município Completo'].unique().tolist()
lista_pares_munic_data = []
for co_muni_ori in list_cod_munic_completo:
    for co_muni_des in list_cod_munic_completo:
        d_={}
        d_['co_muni_ori']=co_muni_ori
        d_['co_muni_des']=co_muni_des
        lista_pares_munic_data.append(d_)
        
df_pares_municipios=pd.DataFrame(lista_pares_munic_data)
df_pares_municipios

CPU times: user 1min 10s, sys: 23.8 s, total: 1min 33s
Wall time: 1min 52s


,co_muni_ori,co_muni_des
0,1100015,1100015
1,1100015,1100379
2,1100015,1100403
3,1100015,1100346
4,1100015,1100023
...,...,...
31024895,5300108,5222005
31024896,5300108,5222054
31024897,5300108,5222203
31024898,5300108,5222302


In [10]:
5570*5570

31024900

## Data air - ANAC

In [11]:
%%time
adjacency_matrices_5570 = {}
# Loop through each month
for month in range(1, 13):
    
    # Filter data for the current month
    df_mes = data_2022.query(f'mes=={month}')[['co_muni_ori', 'co_muni_des','aero_pass']]
    
    # Group by municipality pairs and sum the 'aero_pass' values
    df_agrupado = df_mes.groupby(['co_muni_ori','co_muni_des']).sum().reset_index()
    
    # Merge with df_pares_municipios to ensure all pairs are included
    df_com_5570 = df_agrupado.merge(df_pares_municipios,on=['co_muni_ori','co_muni_des'],how='right')
    
    # Pivot the data to create the adjacency matrix
    matriz_5570 = df_com_5570.pivot_table(index='co_muni_ori', columns='co_muni_des', 
                                          values='aero_pass', aggfunc='sum', fill_value=0)
    
    # Store the resulting matrix in the adjacency_matrices_5570 dictionary
    adjacency_matrices_5570[month] = matriz_5570
    del matriz_5570

CPU times: user 7min 6s, sys: 3min 29s, total: 10min 35s
Wall time: 11min 45s


In [ ]:
# memory MiB used in dataframe of month 1 
print(round(adjacency_matrices_5570[1].memory_usage('deep').sum()/1024**2,2), 'MiB')

In [12]:
for month in range(1, 13):
    print('month',month,
          'lines',adjacency_matrices_5570[month].shape[0],
          'cols',adjacency_matrices_5570[month].shape[1])

month 1 lines 5570 cols 5570
month 2 lines 5570 cols 5570
month 3 lines 5570 cols 5570
month 4 lines 5570 cols 5570
month 5 lines 5570 cols 5570
month 6 lines 5570 cols 5570
month 7 lines 5570 cols 5570
month 8 lines 5570 cols 5570
month 9 lines 5570 cols 5570
month 10 lines 5570 cols 5570
month 11 lines 5570 cols 5570
month 12 lines 5570 cols 5570


In [ ]:
adjacency_matrices_5570[1]

In [ ]:
print(adjacency_matrices_5570[1].loc[1100924,1100189])
print(adjacency_matrices_5570[1].loc[1100189,1100924])

In [ ]:
print(adjacency_matrices_5570[1].loc[5108402,1100122])
print(adjacency_matrices_5570[1].loc[1100122,5108402])

In [ ]:
print(adjacency_matrices_5570[1].loc[1200401,1200203])
print(adjacency_matrices_5570[1].loc[1200203,1200401])

In [13]:
for month in range(1, 13):
    adjacency_matrices_5570[month] = adjacency_matrices_5570[month].reindex(sorted(adjacency_matrices_5570[month].columns), axis=1)

In [14]:
for month in range(1, 13):
    adjacency_matrices_5570[month] = adjacency_matrices_5570[month].sort_index(axis=0)

In [ ]:
# adjacency_matrices_5570[1]

## normalize adjacency matrices to week and capacity (acording to IBGE normalization - bus) 

In [25]:
%%time
# normalize matrices to number of passangers by week
adjacency_matrices_5570_normalized = {}
for month in range(1, 13):
    adjacency_matrices_5570_normalized[month] = adjacency_matrices_5570[month]/4 

CPU times: user 3.68 s, sys: 5.97 s, total: 9.64 s
Wall time: 6.45 s


In [26]:
%%time
# normalize matrices to capacity (instead of number of passangers)
#acording to IBGE normalization bus = 25
adjacency_matrices_5570_normalized2 = {}
for month in range(1, 13):
    adjacency_matrices_5570_normalized2[month] = adjacency_matrices_5570_normalized[month]/25 

CPU times: user 3.4 s, sys: 5.91 s, total: 9.31 s
Wall time: 7.06 s


In [ ]:
print(adjacency_matrices_5570_normalized[1].loc[1200401,1200203])
print(adjacency_matrices_5570_normalized[1].loc[1200203,1200401])

## Road and water - IBGE

In [27]:
# read open data from IBGE
link0= '/Users/andreza/reproducibility_transportation_hubs-early_warning_surveillance_systems/data/fluvi_road_ibge.parquet'

df = pd.read_parquet(link0, engine='pyarrow')

In [28]:
# exclude international conection
df.uf_des.unique()

array(['AM', 'MT', 'PA', 'CE', 'PI', 'RN', 'SE', 'MG', 'GO', 'PB', 'BA',
       'SP', 'PR', 'AP', 'MA', 'RS', 'SC', 'PE', 'TO', 'RJ', 'RO', 'AL',
       'ES', 'DF', 'MS', 'AC', 'RR', 'ARG', 'PAR', 'VEN', 'PER', 'CHI',
       'BOL', 'URU'], dtype=object)

In [29]:
set_lis = ['AM', 'MT', 'PA', 'CE', 'PI', 'RN', 'SE', 'MG', 'BA', 'SP', 'PR',
       'MA', 'AL', 'RS', 'SC', 'PE', 'AP', 'RJ', 'TO', 'GO', 'RO', 'PB',
       'ES', 'AC', 'RR', 'MS', 'DF']

df_road_water = df[df.uf_des.isin(set_lis)]

In [ ]:
df_road_water
#data is in a week format, only need to normalize matrice to month and merge with air matrices

In [ ]:
#select 2022 travels ???
# df_road_water_2022 = df_road_water[df_road_water['ano'] == 2022] #ajustar nome coluna

In [ ]:
# %%time
# adjacency_matrices_5570_road = {}
# # Loop through each month
# for month in range(1, 13):
#     #ajustar nomes colunas
    
#     # Filter data for the current month
#     df_mes = df_road_water[['co_muni_ori', 'co_muni_des', 'tot_conec']].rename(columns={'tot_conec': 'aero_pass'})
    
#     # Group by municipality pairs and sum the 'aero_pass' values
#     df_agrupado = df_mes.groupby(['co_muni_ori','co_muni_des']).sum().reset_index()
    
#     # Merge with df_pares_municipios to ensure all pairs are included
#     df_com_5570 = df_agrupado.merge(df_pares_municipios,on=['co_muni_ori','co_muni_des'],how='right')
    
#     # Pivot the data to create the adjacency matrix
#     matriz_5570 = df_com_5570.pivot_table(index='co_muni_ori', columns='co_muni_des', 
#                                           values='aero_pass', aggfunc='sum', fill_value=0)
    
#     # Store the resulting matrix in the adjacency_matrices_5570 dictionary
#     adjacency_matrices_5570_road[month] = matriz_5570
#     del matriz_5570

In [ ]:
# # Get the number of rows and columns
# for month in range(1, 13):
#     num_rows, num_columns = adjacency_matrices[month].shape
    
#     # Print the number of rows and columns
#     print(f"Month:{month}, Number of rows: {num_rows}, Number of columns: {num_columns}")

# Sum aero, road and waterway matrices

In [30]:
# Read the Adjacent matrix
link2 = '/Users/andreza/reproducibility_transportation_hubs-early_warning_surveillance_systems/code/road_water_adjacency_matrix_symetric.parquet'

adjacency_matrices_5570_road = pd.read_parquet(link2, engine='pyarrow')

In [31]:
adjacency_matrices_5570_road_normalized = adjacency_matrices_5570_road

In [32]:
%%time
adjacency_matrices_5570_complete = {}
for month in range(1, 13):
    adjacency_matrices_5570_complete[month] = adjacency_matrices_5570_road_normalized.add(
        adjacency_matrices_5570_normalized2[month], fill_value=0)

CPU times: user 17.2 s, sys: 9.93 s, total: 27.2 s
Wall time: 26 s


In [ ]:
adjacency_matrices_5570_complete[1]

In [33]:
# Get the number of rows and columns
for month in range(1, 13):
    num_rows, num_columns = adjacency_matrices_5570_complete[month].shape
    
    # Print the number of rows and columns
    print(f"Month:{month}, Number of rows: {num_rows}, Number of columns: {num_columns}")

Month:1, Number of rows: 5570, Number of columns: 5570
Month:2, Number of rows: 5570, Number of columns: 5570
Month:3, Number of rows: 5570, Number of columns: 5570
Month:4, Number of rows: 5570, Number of columns: 5570
Month:5, Number of rows: 5570, Number of columns: 5570
Month:6, Number of rows: 5570, Number of columns: 5570
Month:7, Number of rows: 5570, Number of columns: 5570
Month:8, Number of rows: 5570, Number of columns: 5570
Month:9, Number of rows: 5570, Number of columns: 5570
Month:10, Number of rows: 5570, Number of columns: 5570
Month:11, Number of rows: 5570, Number of columns: 5570
Month:12, Number of rows: 5570, Number of columns: 5570


In [34]:
#save adjacency matrices monthly
for month in range(1, 13):
    table = pa.Table.from_pandas(adjacency_matrices_5570_complete[month])
    pq.write_table(table, f"adjacency_matrix_complete_{month}.parquet")

In [ ]:
print(adjacency_matrices_5570_complete[1].loc[1200401,1200203])
print(adjacency_matrices_5570_complete[1].loc[1200203,1200401])

In [ ]:
print(adjacency_matrices_5570_road_normalized.loc[1200401,1200203])
print(adjacency_matrices_5570_road_normalized.loc[1200203,1200401])

In [ ]:
print(adjacency_matrices_5570_normalized[1].loc[1200401,1200203])
print(adjacency_matrices_5570_normalized[1].loc[1200203,1200401])